In [4]:
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from operator import itemgetter
import numpy as np

In [9]:
def get_data():
    """
    Make a sample classification dataset
    Returns : Independent variable y, dependent variable x
    """
    no_features = 30
    redundant_features = int(0.1*no_features)
    informative_features = int(0.6*no_features)
    repeated_features = int(0.1*no_features)
    x, y = make_classification(n_samples=500,n_features=no_features,flip_y=0.03, n_informative = informative_features, n_redundant = redundant_features,
                               n_repeated = repeated_features,random_state=7)
    return x, y

def build_forest(x, y, x_dev, y_dev):
    """
    Build a random forest of fully grown trees
    and evaluate peformance
    """
    no_tress = 100
    estimator = RandomForestClassifier(n_estimators=no_tress)
    estimator.fit(x,y)
    
    train_predicted = estimator.predict(x)
    train_score = accuracy_score(y, train_predicted)
    dev_predicted = estimator.predict(x_dev)
    dev_score = accuracy_score(y_dev, dev_predicted)
    print("Training Accuracy = %0.2f Dev Accuracy = %0.2f"%(train_score, dev_score))
    
def search_parameters(x, y, x_dev, y_dev):
    """
    Search the parameters of random forest algorithm
    """
    estimator = RandomForestClassifier()
    no_features = x.shape[1]
    no_iterations = 20
    sqr_no_features = int(np.sqrt(no_features))
    parameters = {"n_estimators" : np.random.randint(75,200,no_iterations),
                 "criterion" : ["gini","entropy"],
                  "max_features" : [sqr_no_features,sqr_no_features*2,sqr_no_features*3,sqr_no_features+10]
                 }
    grid = RandomizedSearchCV(estimator=estimator,param_distributions=parameters,
                              verbose=1, n_iter=no_iterations,random_state=77,n_jobs=-1,cv=5)
    grid.fit(x,y)
    print_model_worth(grid, x_dev, y_dev)
    
    return grid.best_estimator_

def print_model_worth(grid, x_dev, y_dev):
    # Print the goodness of the models
    # We take the top 5 models
    scores = sorted(grid.grid_scores_, key=itemgetter(1), reverse=True)[0:5]
    for model_no, score in enumerate(scores):
        print("Model %d, Score=%0.3f")
        print("Parameters = {0}".format(score.parameters))
    print()
    dev_predicted = grid.predict(x_dev)
    print(classification_report(y_dev, dev_predicted))
    
if __name__ == "__main__":
    x, y = get_data()
    
    #Divide the data into Train, dev and test
    x_train, x_test_all, y_train, y_test_all = train_test_split(x, y, test_size = 0.3, random_state = 9)
    x_dev, x_test, y_dev, y_test = train_test_split(x_test_all, y_test_all, test_size = 0.3, random_state=9)
    
    build_forest(x_train, y_train, x_dev, y_dev)
    #get_feature_importance(model)
    

Training Accuracy = 1.00 Dev Accuracy = 0.83
